# 🤖 Amanda - Relationship Support Chatbot

## Demo Setup for Google Colab + ngrok

This notebook sets up the complete Amanda platform in Google Colab and exposes it via ngrok for demonstration purposes.

### What This Notebook Does:
1. ✅ Downloads/Clones the Amanda repository
2. ✅ Installs all dependencies (AI Backend, Backend, Frontend)
3. ✅ Sets up ngrok tunnels for public access
4. ✅ Starts all services (AI Backend, Flask Backend, Frontend)
5. ✅ Provides public URLs for demo

### Prerequisites:
- **ngrok Account**: Sign up at https://ngrok.com (free)
- **ngrok Auth Token**: Get from https://dashboard.ngrok.com/get-started/your-authtoken

### Note:
The AI Backend uses a **simple echo implementation** for testing. Students will implement real AI integration later.

---

## Step 1: Get Repository Files

**Choose ONE of the methods below:**

### Method A: Clone from Public Repository (if repo is public)
Run the cell below if your repository is public.

### Method B: Upload ZIP file (recommended for private repos)
1. Download the repository as ZIP from GitHub
2. Upload it using the file upload cell below

### Method C: Clone with Personal Access Token
Use if repo is private and you have a GitHub PAT.

In [ ]:
import os

# SELECT YOUR METHOD (uncomment ONE option below):

METHOD = 'ZIP'  # Default: Upload ZIP file
# METHOD = 'PUBLIC'  # Use if repo is public
# METHOD = 'TOKEN'  # Use if you have GitHub Personal Access Token

print(f"✅ Selected method: {METHOD}")
print("\nContinue to the next cell for your selected method.")

### Method A: Clone Public Repository

In [ ]:
# Only run this cell if you selected METHOD = 'PUBLIC'

if METHOD == 'PUBLIC':
    if not os.path.exists('amanda'):
        !git clone https://github.com/akhavansafaei/amanda.git
        print("✅ Repository cloned successfully")
    else:
        print("✅ Repository already exists")
        %cd amanda
        !git pull
else:
    print("⏭️  Skipping (not using PUBLIC method)")

### Method B: Upload ZIP File (Recommended)

**Instructions:**
1. Go to your GitHub repository
2. Click **Code** → **Download ZIP**
3. Run the cell below and upload the ZIP file
4. Wait for extraction to complete

In [ ]:
# Only run this cell if you selected METHOD = 'ZIP'

if METHOD == 'ZIP':
    from google.colab import files
    import zipfile
    import shutil
    
    if not os.path.exists('amanda'):
        print("📤 Please upload your amanda repository ZIP file:")
        uploaded = files.upload()
        
        # Get the uploaded file name
        zip_file = list(uploaded.keys())[0]
        print(f"\n📦 Extracting {zip_file}...")
        
        # Extract the ZIP
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall('.')
        
        # GitHub zips have format: repo-branch, so we need to rename
        extracted_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and 'amanda' in d.lower()]
        if extracted_dirs:
            if extracted_dirs[0] != 'amanda':
                shutil.move(extracted_dirs[0], 'amanda')
        
        print("✅ Repository extracted successfully")
    else:
        print("✅ Repository already exists")
else:
    print("⏭️  Skipping (not using ZIP method)")

### Method C: Clone with GitHub Personal Access Token

**Instructions:**
1. Go to GitHub Settings → Developer settings → Personal access tokens
2. Generate new token with 'repo' scope
3. Run the cell below and enter your token

In [ ]:
# Only run this cell if you selected METHOD = 'TOKEN'

if METHOD == 'TOKEN':
    from getpass import getpass
    
    if not os.path.exists('amanda'):
        print("🔐 Enter your GitHub Personal Access Token:")
        github_token = getpass("GitHub PAT: ")
        
        clone_url = f"https://{github_token}@github.com/akhavansafaei/amanda.git"
        !git clone {clone_url}
        print("✅ Repository cloned successfully")
    else:
        print("✅ Repository already exists")
else:
    print("⏭️  Skipping (not using TOKEN method)")

### Verify Repository

In [ ]:
# Verify the repository structure
if os.path.exists('amanda'):
    %cd amanda
    print("✅ Repository is ready!")
    print("\n📁 Repository structure:")
    !ls -la
    print("\n📁 Services:")
    !ls services/
else:
    print("❌ Repository not found. Please run one of the methods above.")

## Step 2: Install Dependencies

Install Python dependencies for AI Backend and Flask Backend.

In [ ]:
# Install AI Backend dependencies
print("📦 Installing AI Backend dependencies...")
!pip install -q -r services/ai_backend/requirements.txt

# Install Backend dependencies
print("📦 Installing Backend dependencies...")
!pip install -q -r services/backend/requirements.txt

# Install ngrok
print("📦 Installing pyngrok...")
!pip install -q pyngrok

print("\n✅ All dependencies installed successfully!")

## Step 3: Configure Environment Variables

⚠️ **IMPORTANT**: Enter your ngrok auth token below:
- **ngrok Auth Token**: From https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
import os
from getpass import getpass

# Get ngrok auth token (required)
print("🔐 Enter your ngrok auth token:")
print("   Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
NGROK_AUTH_TOKEN = getpass("ngrok Auth Token: ")

# Generate a secret key for Flask
import secrets
SECRET_KEY = secrets.token_hex(32)

print("\n✅ Configuration complete!")

## Step 4: Setup ngrok

Configure ngrok to create public tunnels.

In [ ]:
from pyngrok import ngrok, conf

# Set ngrok auth token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("✅ ngrok configured successfully!")

## Step 5: Create Configuration Files

Generate .env files with the proper configuration.

In [ ]:
# Create AI Backend .env (minimal - echo server doesn't need API keys)
ai_backend_env = """# AI Backend Configuration - STARTER VERSION
GRPC_PORT=50051
"""

with open('services/ai_backend/.env', 'w') as f:
    f.write(ai_backend_env)

# Create Backend .env
backend_env = f"""# Flask Backend Configuration
SECRET_KEY={SECRET_KEY}
FLASK_ENV=development
FLASK_HOST=0.0.0.0
FLASK_PORT=5000

# Database
DATABASE_URL=sqlite:///amanda.db

# AI Backend gRPC
GRPC_AI_BACKEND_HOST=localhost
GRPC_AI_BACKEND_PORT=50051

# Session
SESSION_TYPE=filesystem
SESSION_PERMANENT=False
SESSION_USE_SIGNER=True

# CORS (will be updated with ngrok URLs)
CORS_ORIGINS=*
"""

with open('services/backend/.env', 'w') as f:
    f.write(backend_env)

print("✅ Configuration files created!")

## Step 6: Start Services

This will start all three services in the background:
1. AI Backend (gRPC echo server on port 50051)
2. Flask Backend (REST API + WebSocket on port 5000)
3. Frontend (Static files on port 8000)

**Note**: The AI Backend uses a simple echo implementation for testing.

In [ ]:
import subprocess
import time
import threading

# Function to run a service in background
def run_service(name, command, cwd):
    print(f"🚀 Starting {name}...")
    process = subprocess.Popen(
        command,
        shell=True,
        cwd=cwd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    return process

# Start AI Backend (Echo server)
ai_backend_process = run_service(
    "AI Backend (Echo Server)",
    "python server.py",
    "services/ai_backend"
)

# Wait for AI Backend to start
time.sleep(3)
print("✅ AI Backend started on port 50051 (Echo mode for testing)")

# Start Flask Backend
backend_process = run_service(
    "Flask Backend",
    "python app.py",
    "services/backend"
)

# Wait for Backend to start
time.sleep(5)
print("✅ Flask Backend started on port 5000")

# Start Frontend (simple HTTP server)
frontend_process = run_service(
    "Frontend",
    "python -m http.server 8000",
    "services/frontend"
)

# Wait for Frontend to start
time.sleep(2)
print("✅ Frontend started on port 8000")

print("\n🎉 All services are running!")
print("\n⚠️  Note: AI responses will echo your messages (starter version)")
print("   Students will implement real AI integration later.")

## Step 7: Create ngrok Tunnels

Create public URLs for Backend and Frontend.

In [ ]:
# Create tunnel for Backend (port 5000)
backend_tunnel = ngrok.connect(5000, "http")
backend_url = backend_tunnel.public_url

# Create tunnel for Frontend (port 8000)
frontend_tunnel = ngrok.connect(8000, "http")
frontend_url = frontend_tunnel.public_url

print("🌐 Public URLs Created:")
print("="*60)
print(f"Backend API:  {backend_url}")
print(f"Frontend:     {frontend_url}")
print("="*60)

# Store URLs for later use
BACKEND_URL = backend_url
FRONTEND_URL = frontend_url

## Step 8: Update Frontend Configuration

Update the frontend JavaScript to use the ngrok backend URL.

In [ ]:
# Read the original api.js file
with open('services/frontend/static/js/api.js', 'r') as f:
    api_js = f.read()

# Replace localhost with ngrok URL
api_js = api_js.replace(
    "const API_BASE_URL = 'http://localhost:5000';",
    f"const API_BASE_URL = '{BACKEND_URL}';"
)

# Write back
with open('services/frontend/static/js/api.js', 'w') as f:
    f.write(api_js)

# Update websocket.js
with open('services/frontend/static/js/websocket.js', 'r') as f:
    ws_js = f.read()

ws_js = ws_js.replace(
    "const SOCKET_URL = 'http://localhost:5000';",
    f"const SOCKET_URL = '{BACKEND_URL}';"
)

with open('services/frontend/static/js/websocket.js', 'w') as f:
    f.write(ws_js)

print("✅ Frontend configured to use ngrok backend URL")

## 🎉 Step 9: Access Your Application!

Your Amanda platform is now running and accessible via the public URLs above.

### 📱 How to Use:

1. **Open the Frontend URL** (from Step 7)
2. **Sign Up**: Create a new account
3. **Start Chatting**: Send messages and see them echoed back

### 🔗 Quick Access:

In [ ]:
from IPython.display import Markdown, display

display(Markdown(f"""
## 🌐 Your Amanda Demo URLs

### Frontend (Main Application)
**Click here to access Amanda:**  
🔗 [{FRONTEND_URL}]({FRONTEND_URL})

### Backend API
**API Endpoint:**  
🔗 [{BACKEND_URL}]({BACKEND_URL})

### Health Check
**Test if backend is running:**  
🔗 [{BACKEND_URL}/health]({BACKEND_URL}/health)

---

### 📋 Demo Checklist:
- [ ] Open Frontend URL
- [ ] Create account (signup)
- [ ] Login
- [ ] Create a new chat
- [ ] Send a message
- [ ] See echo response (starter AI backend)

### ⚠️ Important Notes:
- The AI Backend uses a **simple echo implementation** for testing
- Students will implement real AI integration (Claude, GPT, etc.) later
- These URLs are **temporary** and will expire when you stop this Colab session
- ngrok free tier URLs are **public** - anyone with the link can access
- If you see an ngrok warning page, click "Visit Site"
"""))

## 🔍 Step 10: Monitor Services (Optional)

Check if all services are running properly.

In [ ]:
import requests

# Check Backend health
try:
    response = requests.get(f"{BACKEND_URL}/health", timeout=5)
    if response.status_code == 200:
        print("✅ Backend is healthy:", response.json())
    else:
        print("⚠️ Backend returned:", response.status_code)
except Exception as e:
    print("❌ Backend health check failed:", e)

# Check Frontend
try:
    response = requests.get(FRONTEND_URL, timeout=5)
    if response.status_code == 200:
        print("✅ Frontend is accessible")
    else:
        print("⚠️ Frontend returned:", response.status_code)
except Exception as e:
    print("❌ Frontend check failed:", e)

print("\n📊 All checks complete!")

## 🔧 Troubleshooting

### Service Not Starting?
Run the cell below to check service status and logs.

In [ ]:
# Check if processes are running
print("Service Status:")
print("="*60)

if ai_backend_process.poll() is None:
    print("✅ AI Backend: Running")
else:
    print("❌ AI Backend: Stopped")
    print("Error:", ai_backend_process.stderr.read())

if backend_process.poll() is None:
    print("✅ Flask Backend: Running")
else:
    print("❌ Flask Backend: Stopped")
    print("Error:", backend_process.stderr.read())

if frontend_process.poll() is None:
    print("✅ Frontend: Running")
else:
    print("❌ Frontend: Stopped")
    print("Error:", frontend_process.stderr.read())

print("="*60)

# Common fixes
print("\n💡 Common Fixes:")
print("1. Wait a few seconds for services to fully start")
print("2. Re-run Step 6 if any service crashed")
print("3. Check that ports are not already in use")

## 🔄 Restart Services (If Needed)

If services crash, run this cell to restart everything.

In [ ]:
# Kill existing processes
try:
    ai_backend_process.kill()
    backend_process.kill()
    frontend_process.kill()
except:
    pass

print("🔄 Restarting services...")
time.sleep(2)

# Restart everything
ai_backend_process = run_service("AI Backend (Echo Server)", "python server.py", "services/ai_backend")
time.sleep(3)

backend_process = run_service("Flask Backend", "python app.py", "services/backend")
time.sleep(5)

frontend_process = run_service("Frontend", "python -m http.server 8000", "services/frontend")
time.sleep(2)

print("✅ Services restarted!")

## 🧹 Cleanup (Run Before Stopping)

Run this cell to cleanly stop all services and close ngrok tunnels.

In [ ]:
print("🧹 Cleaning up...")

# Kill processes
try:
    ai_backend_process.kill()
    print("✅ AI Backend stopped")
except:
    pass

try:
    backend_process.kill()
    print("✅ Flask Backend stopped")
except:
    pass

try:
    frontend_process.kill()
    print("✅ Frontend stopped")
except:
    pass

# Close ngrok tunnels
try:
    ngrok.kill()
    print("✅ ngrok tunnels closed")
except:
    pass

print("\n🎉 Cleanup complete!")